In [4]:
from docx import Document

def extract_text_from_docx(path):
    doc = Document(path)
    return "\n".join([para.text for para in doc.paragraphs if para.text.strip()])

doc1 = extract_text_from_docx('1747395490801_PMT Pro User FAQ.docx')
doc2 = extract_text_from_docx('1747395531476_Status and Evaluation on 15MAY2025 - PM Pro.docx')

raw_knowledge = doc1 + "\n" + doc2
len(raw_knowledge)

34442

In [6]:
import textwrap

def chunk_text(text, chunk_size=300, overlap=50):
    words = text.split()
    chunks = []
    start = 0
    while start < len(words):
        end = min(start + chunk_size, len(words))
        chunk = " ".join(words[start:end])
        chunks.append(chunk)
        start += chunk_size - overlap
    return chunks

chunks = chunk_text(raw_knowledge)
len(chunks)

21

In [7]:
import requests

def get_ollama_embedding(text, model="mxbai-embed-large"):
    response = requests.post(
        "http://localhost:11434/api/embeddings",
        json={"model": model, "prompt": text}
    )
    return response.json()["embedding"]

chunk_embeddings = [get_ollama_embedding(chunk) for chunk in chunks]


In [10]:
len(chunk_embeddings[0])

1024

In [11]:
len(chunk_embeddings)

21

In [13]:
import chromadb

# Use PersistentClient in v0.4+
chroma_client = chromadb.PersistentClient(path="./rag_chroma")
collection = chroma_client.get_or_create_collection("pmt_pro_docs")

for i, (chunk, embedding) in enumerate(zip(chunks, chunk_embeddings)):
    collection.add(
        documents=[chunk],
        embeddings=[embedding],
        ids=[f"chunk_{i}"]
    )


In [14]:
def get_query_embedding(query):
    return get_ollama_embedding(query)

def retrieve_context(query, top_k=3):
    query_embedding = get_query_embedding(query)
    results = collection.query(query_embeddings=[query_embedding], n_results=top_k)
    return "\n---\n".join(results['documents'][0])

query = "How does PMT Pro handle overdue tasks?"
retrieved_context = retrieve_context(query)
print(retrieved_context)


PMT Pro - User FAQ (Frequently Asked Questions) GENERAL USAGE Q1: What is PMT Pro? A1: PMT Pro is an AI-powered project management tool designed to help teams plan, collaborate, and execute projects more efficiently. It integrates traditional project management features like Kanban boards, task lists, timelines, and team chat with advanced AI capabilities including smart task prioritization, predictive analytics, and an expert chatbot assistant, along with on-tip-availability of everything about the project using Agentic RAG technology for keeping all informed with relevant information, for the ask. Q2: How is PMT Pro different from Basecamp, Trello, or Asana? A2: Unlike those platforms, PMT Pro includes: A 24/7 AI assistant powered by a Retrieval-Augmented Generation (RAG) model Automatic task prioritization and deadline tracking Dynamic filtering by project context Meeting summaries, intelligent suggestions, and risk alerts A built-in document and link repository for each project Q3:

In [15]:
def generate_answer(query, context):
    prompt = f"""You are a helpful assistant for PMT Pro. Use the context below to answer the user's question.\n\nContext:\n{context}\n\nQuestion: {query}\nAnswer:"""
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={"model": "tinyllama:1.1b", "prompt": prompt, "stream": False}
    )
    return response.json()["response"]

answer = generate_answer(query, retrieved_context)
print(answer)


PMT Pro addresses overdue tasks in its to-do list, which is automatically sorted by due date. In the overdue task list, overdue tasks are highlighted at the top with an "overdue" label indicating they need to be addressed soon. Additionally, PMT Pro allows users to view all tasks on a Day/Week/Month timeframe and filter by project or team member. Menctions (e.g., @john) can be used to turn overdue chat messages into tasks with due dates and assignment using natural language commands. The AI assistant also provides real-time collaboration between members and can parse chat content, turn overdue tasks into tasks with due dates and assignments using natural language commands, and summarize recent project activity, prioritize your To-Do list, answer project-specific questions, suggest next steps or highlight risks, turn notes or chat into tasks, and generate accurate, up-to-date, and project-specific guidance.


In [17]:
from IPython.display import Markdown, display

In [18]:
display(Markdown(answer))

PMT Pro addresses overdue tasks in its to-do list, which is automatically sorted by due date. In the overdue task list, overdue tasks are highlighted at the top with an "overdue" label indicating they need to be addressed soon. Additionally, PMT Pro allows users to view all tasks on a Day/Week/Month timeframe and filter by project or team member. Menctions (e.g., @john) can be used to turn overdue chat messages into tasks with due dates and assignment using natural language commands. The AI assistant also provides real-time collaboration between members and can parse chat content, turn overdue tasks into tasks with due dates and assignments using natural language commands, and summarize recent project activity, prioritize your To-Do list, answer project-specific questions, suggest next steps or highlight risks, turn notes or chat into tasks, and generate accurate, up-to-date, and project-specific guidance.

In [19]:
query = "How do I use the Kanban board?"
retrieved_context = retrieve_context(query)

answer = generate_answer(query, retrieved_context)
display(Markdown(answer))

To use the Kanban board in PMT Pro, follow these steps:
1. Open the project dashboard and go to "Kanban board" on the left sidebar.
2. Click on the plus sign (+) to add a new card or click on an existing card to update it.
3. Choose the color-coded priority of each task (red for high priority, orange for medium priority, green for low priority), drag and drop cards between stages, assign due dates, and add team members.
4. You can also use the Kanban cards as a way to quickly scan through tasks and get a visual overview.
5. To jump into meetings smoothly, PMT Pro offers an integrated AI assistant called the "Chatbot" that can greet users and offer help. It can prioritize tasks based on project history and goals, serve as an on-demand project analyst for teamleads, and provide recommendations based on past interactions with the team file-lzwt1a8jfgwduo7egwmbrrfile-lzwt1a8jfgwduo7egwmbrr.
6. PMT Pro offers a smart to-do list with priority order and tags for easy task organization, including tasks pulled from the Kanban board versus ad-hoc activities entered via the general task page. By tagging overdue tasks in red, you can focus on what's important right now and reduce the chaos of missed deadlines.
7. Important files and links are organized in a centralized document repository.
8. To get the most out of PMT Pro, try expanding collapsed sections to see all relevant information at once, and tailor views by collapsing sections not in use to increase focus and usability.

In [20]:
query = "How does team chat work in PMT Pro?"
retrieved_context = retrieve_context(query)

answer = generate_answer(query, retrieved_context)
display(Markdown(answer))

In PMT Pro, the integrated team chat is not just a communication channel but also linked to project data. The AI can reference chat context when providing answers or suggestions and team conversation can be quickly turned into linked tasks or knowledge entries. This integration of collaboration and work tracking sets PMT Pro apart from other project management tools like Trello (which has no native chat) and Asana (which has comments but no real-time chat).

In [21]:
query = "Can the chatbot be customized or trained further?"
retrieved_context = retrieve_context(query)

answer = generate_answer(query, retrieved_context)
display(Markdown(answer))

Sure, the chatbot in PMT Pro can be customized and trained further through future updates that improve the platform's capabilities and user experience. The team can upload their project goals, style preferences, and voice tuning to further enhance the chatbot's abilities and offer more personalized support to users. These upgrades will help PMT Pro to remain a top-of-the-line project management tool and provide better support for distributed teams.

In [22]:
query = "What if I find a bug or have a feature request?"
retrieved_context = retrieve_context(query)

answer = generate_answer(query, retrieved_context)
display(Markdown(answer))

To report bugs or make suggestions, users can use the chatbot or contact their admin to report issues or suggest features. PMT Pro continuously improves as more documents and chat context are added, and future updates may include custom project goals, style preference, and voice tuning for improved personalized support. The investor-oriented look at how PMT Pro stands out in a crowded market includes Executive Overview: "PMT Pro is a next-generation project management tool that combines robust task management with artificial intelligence to deliver proactive planning, collaboration, and automaton capabilities." This AI-powered platform functions as a project manager co-pilot who empowers teams to work smarter and faster. PMT Pro addresses key pain points seen in traditional tools like Basecamp and Asana and offers a unified solution that matches the core strengths of popular tools like Basecamp and Asana. Receent Enhancement's (2025) include expanding top navigation to allow for more context-sensitive suggestions, an improved UI, and persistent context that keeps distributed teams connected. The chatbot and integrated AI ChaTBot are designed to provide ongoing support and assistance throughout the project lifecycle.

In [23]:
query = "List the applications that PMT Pro can handle."
retrieved_context = retrieve_context(query)

answer = generate_answer(query, retrieved_context)
display(Markdown(answer))

Sure! Here are some examples of other AI-powered project management tools PMT Pro can handle, based on their respective features and user experience:

1. Basecamp
2. Trello
3. Asana
4. Kanbanboard (Taskboard, ToDo list, Project Kanban)
5. Agile software teams
6. Distributed teams
7. Managing multiple concurrent projects
8. Task Prioritization
9. Automatic task prioritization and deadline tracking
10. Dynamic filtering by project context 
11. Meeting summaries, intelligent suggestions, and risk alerts
12. Built-in document and link repository for each project
13. Kanban boards displaying tasks in four stages: To Do, In Progress, Review, and Done
14. Priority tags
15. Color-coded priority
16. Unified workspace - users have a single application that consolidates project planning, execution, and communication
17. Concisely separating concerns within one screen for ease of context continuity with the AI
18. Clean separation of concerns within one screen for easy team coordination
19. Intuitive interface with reduced clutter - users can show or hide panel sections based on their current focus
20. Properly categorized files and knowledge at hand to improve team efficiency and quality as well as a project knowledge base

In [24]:
query = "How do we prioritize the task on PMT?"
retrieved_context = retrieve_context(query)

answer = generate_answer(query, retrieved_context)
display(Markdown(answer))

The answer to the user's question is: To prioritize tasks in PMT Pro, the chatbot uses the RAG (Retrieval-Augmenteed Generation) model that automatically ranks and highlights tasks based on urgency, deadlines, and even dependencies. The chatbot analyzes project data and user behavior to determine which task should be given a higher priority based on its contextual relevance. Users can ask the chatbot to generate accurate, tailored responses based on their specific needs, as well as ask it to categorize tasks by category, such as "To Do" or "Kanban cards," and rank them accordingly. By leveraging the AI capabilities of the chatbot, PMT Pro provides a more efficient and effective way for teams to plan, collaborate, and execute projects.

In [26]:
query = "Give a complete overview of this PMT. What it is for and what it isn't for? Give the response in bullet points."
retrieved_context = retrieve_context(query)

answer = generate_answer(query, retrieved_context)
display(Markdown(answer))

Sure! Here's a detailed overview of PMT Pro:

1. AI-powered project management tool designed to help teams plan, collaborate, and execute projects more efficiently.
2. Integrates traditional project management features like Kanban boards, task lists, timelines, and team chat with advanced AI capabilities including smart task prioritization, predictive analytics, and an expert chatbot assistant, along with on-time-availability of everything about the project using Agentic RAG technology for keeping all informed with relevant information, for the ask.
3. Different levels of access based on user roles: PMT Pro is designed to be simple enough for daily users and powerful enough for technical and project leads.
4. Kanban boards display tasks in four stages (To Do, In Progress, Review, Done) and users can drag and drop cards between stages, assign due dates, add team members, and get team progress updates.
5. Used by over 20,000 teams worldwide.
6. User-friendly UI with a built-in Kanban board, chatbox, and task summaries.
7. Integrated chatbot that can answer frequently asked questions (FAQs) and perform smart tasks like prioritizing tasks based on priority levels or automating tasks such as email templates and file downloads.
8. Team management tools include:
9. Real-time communication through video calls, instant messaging, team chat, and voice/video conferencing (VC) for remote teams.
10. Integrated document storage with integration with popular document repositories like Google Drive or Dropbox.
11. Assistant for PMT Pro users. It can perform smart task prioritization, predictive analytics, and provide automation of tasks like email templates, file downloads, and meeting summaries based on team member status, project status, and other data sources.
12. Integrated with social media platforms to enable real-time communication between teams.
13. Advanced features for collaboration such as:
14. Integration with popular project management software, including Asana, Basecamp, Trello, and more.
15. Integration with AI-driven project portfolios that provide insights into team and project performance over time.
16. Automated communication to avoid unnecessary meetings or stand-ups.